In [1]:
import numpy as np
import random
import csv
import mido
from mido import MidiFile
import os
import glob

In [15]:
## remove any files generated previously

files = glob.glob('data/*')
for f in files:
    os.remove(f)

In [2]:
# the number of patterns that is being used - this should be input by the user
num_patterns = 3

In [3]:
# read midi file, convert it into the matrix representation, print as csv
# when the met condition is true, the metadata needed for subsequence operatoins is written to file

def generate_csv(filename, output_csv, met):
 mid = MidiFile(filename)
     
 if mid.tracks[0][1].type == 'set_tempo': 
  tempo = mid.tracks[0][1].tempo
  bpm = round(1000000*60/mid.tracks[0][1].tempo)
 else:
  tempo = mid.tracks[0][2].tempo
  bpm = round(1000000*60/mid.tracks[0][2].tempo)
 tpb = mid.ticks_per_beat

 # tolerances 
 t_ptc = 0
 t_amp = round(127*5/100)
 t_dur = (60/bpm)*0.05

 seq = []
 track = mid.tracks[0]
 for i in range (0, len(track)):
  if (track[i].type=='note_on'):        
   if(track[i].time > 0):
    ptc = 0
    amp = 0
    dur = mido.tick2second(track[i].time, tpb, tempo)
    seq.append([ptc, amp, dur])        
   ptc = track[i].note
   amp = track[i].velocity
   dur = mido.tick2second(track[i+1].time, tpb, tempo)
   seq.append([ptc, amp, dur])

   train_pat = np.asarray(seq)
   with open(output_csv, mode='w', newline='') as f:
    data_writer = csv.writer(f)    
    for i in range(0, len(train_pat)):
     tt = []
     for j in range(0, len(train_pat[i])):
      tt.append(train_pat[i][j])
     data_writer.writerow(tt) 

 if met == 1:
  with open('data/metadata.csv', mode='w', newline='') as f:
   data_writer = csv.writer(f)
   data_writer.writerow([tempo])
   data_writer.writerow([bpm])
   data_writer.writerow([tpb])
   data_writer.writerow([t_ptc])
   data_writer.writerow([t_amp])
   data_writer.writerow([t_dur])
   data_writer.writerow([num_patterns])

In [4]:
# call the function for the midi track, and each pattern
# metadata parameter is only true for the midi track. It can be true for either 

generate_csv('midi/Nikolai Rimsky-Korsakov - Flight Of The Bumblebee.mid', 'data/test_data.csv', 1)
generate_csv('midi/Nikolai Rimsky-Korsakov - Flight Of The Bumblebee-p1.mid', 'data/pat_0.csv', 0)
generate_csv('midi/Nikolai Rimsky-Korsakov - Flight Of The Bumblebee-p2.mid', 'data/pat_1.csv', 0)
generate_csv('midi/Nikolai Rimsky-Korsakov - Flight Of The Bumblebee-p3.mid', 'data/pat_2.csv', 0)